In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 6.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 36.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-wl_bnj2g/overlay/local/bin
    changing mode of /tmp/pip-build-env-wl_bnj2g/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-wl_bnj2g/overlay/lo

In [2]:
!pip install huggingface_hub
!pip -q install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00


In [3]:
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [4]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096, # Context window
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    temperature = 0,
)



AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [5]:
import pandas as pd
import re

csv_path = "/content/listings.csv"
df = pd.read_csv(csv_path)

In [47]:
TEMPLATE = """
''SYSTEM: Estás trabajando con un dataframe de pandas en Python. El nombre del dataframe es `df` y contiene un listado
de casas o departamentos en renta o en venta. Está formado por las columnas siguientes:
-'id'
-'listing_type': el valor de esta columna indica si la propiedad listada esta en venta o en renta, sus valores posibles son 'for-sale' y 'for-rent'
-'property_type': esta columna indica si la propiedad es casa o departtamento, toma solo dos valores: 'apartment' o 'house'.
-'last_price': precio con el que se vendió o rentó la propiedad.
-'num_bedrooms': numero de cuartos/recámaras.
-'num_bathrooms': numero de baños
-'has_pool': 1 si tiene alberca, 0 si no.
'has_terrace': 1 si tiene terraza, 0 si no.
'surface_total': superficie total de la propiedad, es un valor numérico.

Te haré preguntas sobre el dataframe y debes contestarme incluyendo el codigo de python entre `. Por ejemplo si te pregunto
cuál es el promedio del precio de departamentos con un baño me deberás responder así:

El promedio del precio de departamentos con un baño es `df[(df['property_type'] == 'apartment') & (df['num_bathrooms'] == 1)]['last_price'].mean()`.
Tienes que repetir toda la información que se pidió en la pregunta y dar luego el codigo sin explicarlo.
No olvides usar parentesis entre las condiciones si usas el simbolo &.
USER: {question}
"""


In [7]:
prompt = PromptTemplate(template=TEMPLATE, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [8]:
question = "¿Cuál es el precio promedio de los departamentos en venta?"
resp = llm_chain.run(question)
parts = re.split("`", resp)
answer = parts[0][8:] + str(eval(parts[1])) + "."

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


SYSTEM: El precio promedio de los departamentos en venta es `df[(df['listing_type'] == 'for-sale') & (df['property_type'] == 'apartment')]['last_price'].mean()`.


In [9]:
print(answer)

El precio promedio de los departamentos en venta es 6876558.794805228.


In [69]:
prompt = PromptTemplate(template=TEMPLATE, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "¿Cuántos departamentos están a la renta?"
resp = llm_chain.run(question)
parts = re.split("`", resp)
answer = parts[0][8:] + str(eval(parts[1])) + "."

Llama.generate: prefix-match hit


SYSTEM: El número de departamentos en renta es `df[(df['listing_type'] == 'for-rent')]['id'].count()`.


In [70]:
print(answer)

El número de departamentos en renta es 772.


In [52]:
prompt = PromptTemplate(template=TEMPLATE, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "¿Cuántos departamentos tienen cuatro cuartos?"
resp = llm_chain.run(question)
parts = re.split("`", resp)
answer = parts[0][8:] + str(eval(parts[1])) + "."

Llama.generate: prefix-match hit


SYSTEM: El número de departamentos con cuatro cuartos es `df[(df['property_type'] == 'apartment') & (df['num_bedrooms'] == 4)]`.

Si bien este modelo contesta bien varias preguntas, en otras le falta hacer un ultimo paso, por ejemplo aplicar la función len() de pandas.

In [51]:
eval("df[(df['property_type'] == 'apartment') & (df['num_bedrooms'] == 4)]")

,id,listing_type,property_type,last_price,num_bedrooms,num_bathrooms,has_pool,has_terrace,surface_total
280,1082631d4d1167873c903bb180a37541,for-sale,apartment,6500000.0,4.0,1.0,NaN,NaN,135.00
384,b0eed5a7f58f2bbb46a02b6f05ec12f4,for-sale,apartment,4745000.0,4.0,2.0,NaN,NaN,97.00
403,f2445e3a2ef4f90ee39555508deee520,for-sale,apartment,6500000.0,4.0,1.0,NaN,NaN,135.00
942,cc4d6626c1555214e9b4b7b00347f2eb,for-sale,apartment,6500000.0,4.0,2.0,NaN,NaN,139.00
1038,6120251dd4411091ab8f3cd90acee573,for-sale,apartment,4400000.0,4.0,1.0,NaN,NaN,105.00
1131,30cc6c390d5a3449be2048638e43f2fc,for-rent,apartment,68000.0,4.0,3.0,NaN,1.0,300.00
1355,019fb7a81564d4f2f161e65d9d07d553,for-rent,apartment,68000.0,4.0,3.0,NaN,1.0,300.00
1482,cbdc5c18ea592c0c0e15c5a003b21cea,for-sale,apartment,3650000.0,4.0,2.0,NaN,NaN,65.55
2938,d5ace8b815976a9111325e7188e31700,for-rent,apartment,30000.0,4.0,1.0,NaN,NaN,120.00
2986,65ebba52b6ac52259fea3fa3805f0f9c,for-rent,apartment,30000.0,4.0,1.0,NaN,NaN,120.00
